In [ ]:
import pandas as pd
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from google.colab import drive
import csv

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow as tf

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path=os.path.join(os.getcwd(),'drive','MyDrive','Colab Notebooks','sensor.csv')

In [ ]:
input_data=pd.read_csv(file_path)

In [ ]:
for column in input_data.columns:
  if 'sensor' in column:
    try:
      temp_mean=input_data[column].mean()
    except:
      temp_mean=0
    if np.isnan(temp_mean):
      temp_mean=0
    
    input_data[column].replace(np.nan,temp_mean,inplace=True)

In [ ]:
print(input_data.columns)

Index(['Unnamed: 0', 'timestamp', 'sensor_00', 'sensor_01', 'sensor_02',
       'sensor_03', 'sensor_04', 'sensor_05', 'sensor_06', 'sensor_07',
       'sensor_08', 'sensor_09', 'sensor_10', 'sensor_11', 'sensor_12',
       'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17',
       'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21', 'sensor_22',
       'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27',
       'sensor_28', 'sensor_29', 'sensor_30', 'sensor_31', 'sensor_32',
       'sensor_33', 'sensor_34', 'sensor_35', 'sensor_36', 'sensor_37',
       'sensor_38', 'sensor_39', 'sensor_40', 'sensor_41', 'sensor_42',
       'sensor_43', 'sensor_44', 'sensor_45', 'sensor_46', 'sensor_47',
       'sensor_48', 'sensor_49', 'sensor_50', 'sensor_51', 'machine_status'],
      dtype='object')


In [ ]:
input_data.drop(['Unnamed: 0', 'timestamp','sensor_00','sensor_15','sensor_50','sensor_51'],axis=1, inplace=True)


In [ ]:
print(set(list(input_data['machine_status'])))

{'BROKEN', 'NORMAL', 'RECOVERING'}


In [ ]:
broken_input_data=input_data[input_data['machine_status']=='BROKEN']
print(len(broken_input_data))
broken_input_data.drop(['machine_status'],inplace=True,axis=1)

7


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
normal_input_data=input_data[input_data['machine_status']=='NORMAL']
normal_input_data.drop(['machine_status'],inplace=True,axis=1)
#train,test=normal_input_data[0:1500],normal_input_data[1500:2000]
#print(len(train),len(test))

1500 500


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
recovering_input_data=input_data[input_data['machine_status']=='RECOVERING']
print(len(recovering_input_data))
recovering_input_data.drop(['machine_status'],inplace=True,axis=1)
train,test=recovering_input_data[0:1500],recovering_input_data[1500:2000]
print(len(train),len(test))

14477
1500 500


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [ ]:
n_past = 5
n_future = 1 
n_features = 48

In [ ]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))

X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [ ]:
print(X_train.shape)
print(y_train.shape)

(1495, 5, 48)
(1495, 1, 48)


In [ ]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)
model_e1d1.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 5, 48)]      0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 100), (None, 59600       input_3[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 1, 100)       0           lstm_4[0][0]                     
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 1, 100)       80400       repeat_vector_2[0][0]            
                                                                 lstm_4[0][1]               

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

In [ ]:
pred_e1d1=model_e1d1.predict(X_test)

In [ ]:
print(pred_e1d1[0])

[[4.1734745e+01 4.3129295e+01 4.0654800e+01 1.2863489e+01 1.1888101e-01
  6.3148184e+00 1.1900096e+01 9.5278292e+00 1.4840310e+01 2.3716507e+00
  1.4732517e+00 1.2888451e-01 2.5965430e-02 4.2996292e+01 4.2421227e+01
  4.3552860e+01 2.4625964e+00 4.3545914e+01 4.1974079e+01 4.0633976e+01
  4.3304276e+01 4.3799389e+01 4.3162777e+01 4.2306080e+01 4.1181667e+01
  4.2532051e+01 4.3621494e+01 4.3407742e+01 4.0777336e+01 4.3231487e+01
  4.2919220e+01 4.2446232e+01 4.2245571e+01 4.0949406e+01 4.1688931e+01
  4.1582069e+01 3.4124012e+01 2.9990427e+01 3.6432755e+01 2.8895042e+01
  3.1761101e+01 3.6235302e+01 3.3810612e+01 3.3951614e+01 3.4363174e+01
  3.3705082e+01 3.5546787e+01 3.5384823e+01]]


In [ ]:
import pickle
loaded_model = pickle.load(open('random_forest.sav', 'rb'))
for i in range(0,len(pred_e1d1)):
  print(loaded_model.predict(pred_e1d1[i]))


['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']
['NORMAL']